
# 3.0 — Data Preprocessing (Bootcamp CDIA)

**Objetivo:** preparar os dados limpos (`bootcamp_train_processed.csv`) para modelagem em manutenção preditiva (multi‑rótulo).  
**Metodologia:** CRISP‑DM — etapa de *Data Preparation* focada em: encoding, escala, balanceamento, engenharia de atributos e pipelines reprodutíveis.

> Arquivo esperado em: `/mnt/data/bootcamp_train_processed.csv`.


In [ ]:

# === Imports ===
import os
import numpy as np
import pandas as pd

# sklearn
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

# Optional: imbalanced-learn (se disponível)
try:
    from imblearn.over_sampling import SMOTE
    IMBLEARN_AVAILABLE = True
except Exception:
    IMBLEARN_AVAILABLE = False

RANDOM_STATE = 42
pd.set_option("display.max_columns", 200)
print("Libraries loaded. imbalanced-learn available:", IMBLEARN_AVAILABLE)


## 1) Carregar dados limpos

In [ ]:

# Caminho do dataset limpo
DATA_PATH = "/mnt/data/bootcamp_train_processed.csv"

assert os.path.exists(DATA_PATH), f"Arquivo não encontrado: {DATA_PATH}"
df = pd.read_csv(DATA_PATH)

print("Shape:", df.shape)
display(df.head(3))
display(df.describe(include='all').T.head(12))


## 2) Definir variáveis‑alvo (multi‑rótulo) e features

In [ ]:

# Alvos esperados conforme enunciado do desafio
expected_targets = ['falha_maquina', 'FDF', 'FDC', 'FP', 'FTE', 'FA']
target_cols = [c for c in expected_targets if c in df.columns]

# Colunas categóricas/numéricas
categorical_features = [c for c in df.columns if df[c].dtype == 'object']
# Remove alvos e IDs da lista
id_like = [c for c in df.columns if 'id' in c.lower()]
numeric_features = [c for c in df.select_dtypes(include=[np.number]).columns if c not in target_cols + id_like]

# Remover 'tipo' das numéricas se estiver como object por engano
if 'tipo' in df.columns and df['tipo'].dtype == 'object' and 'tipo' not in categorical_features:
    categorical_features.append('tipo')
if 'tipo' in numeric_features and df['tipo'].dtype != np.number:
    numeric_features.remove('tipo')

print("Targets:", target_cols)
print("Categóricas:", categorical_features)
print("Numéricas:", numeric_features[:10], "...")


## 3) Funções utilitárias

In [ ]:

def label_distribution(y_df):
    dist = {}
    for c in y_df.columns:
        vc = y_df[c].value_counts(dropna=False).to_dict()
        pos = vc.get(1, 0)
        neg = vc.get(0, 0)
        dist[c] = {"positivos": int(pos), "negativos": int(neg), "pct_pos": round(100*pos/(pos+neg+1e-9),2)}
    return pd.DataFrame(dist).T.sort_index()

def choose_scaler(name='standard'):
    name = (name or '').lower()
    if name == 'minmax':
        return MinMaxScaler()
    if name == 'robust':
        return RobustScaler()
    return StandardScaler()  # default

def add_feature_engineering(df_in: pd.DataFrame) -> pd.DataFrame:
    df_out = df_in.copy()
    # Exemplos de features derivadas (robustas a valores inválidos)
    if set(['temperatura_processo','temperatura_ar']).issubset(df_out.columns):
        with np.errstate(divide='ignore', invalid='ignore'):
            df_out['ratio_temp_proc_ar'] = df_out['temperatura_processo'] / df_out['temperatura_ar']
    if set(['torque','velocidade_rotacional']).issubset(df_out.columns):
        df_out['potencia_proxy'] = df_out['torque'] * df_out['velocidade_rotacional']
    # Tratar inf e NaN gerados
    df_out.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_out.fillna(df_out.median(numeric_only=True), inplace=True)
    return df_out


## 4) (Opcional) Aplicar engenharia de atributos

In [ ]:

APPLY_FEATURE_ENGINEERING = True
X_full = df.drop(columns=[c for c in target_cols])
if APPLY_FEATURE_ENGINEERING:
    X_full = add_feature_engineering(X_full)

y_full = df[target_cols].astype(int) if target_cols else pd.DataFrame()

print("X_full shape:", X_full.shape, "| y_full shape:", y_full.shape)
display(label_distribution(y_full))


## 5) Split treino/teste

In [ ]:

# Para multilabel, não há estratificação nativa simples -> usaremos split aleatório fixo
X_train, X_test, y_train, y_test = train_test_split(
    X_full, y_full, test_size=0.2, random_state=RANDOM_STATE
)
print("Train:", X_train.shape, y_train.shape, "| Test:", X_test.shape, y_test.shape)


## 6) Preprocessamento (One‑Hot + Scaler) com Pipeline

In [ ]:

SCALER_NAME = 'standard'  # opções: 'standard', 'minmax', 'robust'
scaler = choose_scaler(SCALER_NAME)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, [c for c in X_train.columns if c in numeric_features or np.issubdtype(X_train[c].dtype, np.number)]),
        ('cat', OneHotEncoder(handle_unknown='ignore'), [c for c in X_train.columns if c in categorical_features])
    ],
    
)

preprocessor


## 7) Abordagens para desbalanceamento

In [ ]:

USE_SMOTE = False and IMBLEARN_AVAILABLE  # altere para True se quiser testar SMOTE (se disponível)

if USE_SMOTE and IMBLEARN_AVAILABLE:
    print("SMOTE será aplicado dentro de um fluxo manual após o fit_transform do preprocessor.")
else:
    print("Usando class_weight='balanced' no modelo (recomendado como baseline).")


## 8) Pipeline de modelagem (baseline: Logistic Regression One‑Vs‑Rest)

In [ ]:

# Modelo base
base_estimator = LogisticRegression(max_iter=200, class_weight='balanced', solver='lbfgs', n_jobs=None)
clf = OneVsRestClassifier(base_estimator, n_jobs=None)

pipeline = Pipeline(steps=[
    ('prep', preprocessor),
    ('clf', clf)
])

pipeline


## 9) Treinamento e avaliação rápida

In [ ]:

pipeline.fit(X_train, y_train)

# Predições
y_pred = pipeline.predict(X_test)
y_proba = pipeline.decision_function(X_test) if hasattr(pipeline.named_steps['clf'], "decision_function") else pipeline.predict_proba(X_test)

# Relatório por classe
for i, col in enumerate(y_test.columns):
    print(f"\n=== Classe: {col} ===")
    print(classification_report(y_test[col], y_pred[:, i]))
    try:
        # Para multilabel binário, podemos calcular ROC AUC por classe (se proba disponível)
        if isinstance(y_proba, list):  # alguns estimadores retornam lista por classe
            proba_i = y_proba[i][:,1] if y_proba[i].ndim==2 else y_proba[i]
        else:
            # decision_function pode retornar score contínuo; roc_auc aceita
            proba_i = y_proba[:, i]
        auc = roc_auc_score(y_test[col], proba_i)
        print("ROC AUC:", round(auc, 4))
    except Exception as e:
        print("ROC AUC não disponível:", e)


## 10) (Opcional) Exportar dados pré‑processados

In [ ]:

EXPORT_TRANSFORMED = False

if EXPORT_TRANSFORMED:
    Xtr = pipeline.named_steps['prep'].fit_transform(X_train)
    Xte = pipeline.named_steps['prep'].transform(X_test)
    np.save('/mnt/data/X_train_preprocessed.npy', Xtr)
    np.save('/mnt/data/X_test_preprocessed.npy', Xte)
    y_train.to_csv('/mnt/data/y_train.csv', index=False)
    y_test.to_csv('/mnt/data/y_test.csv', index=False)
    print("Arquivos salvos em /mnt/data/*.npy e /mnt/data/*.csv")



## 11) Próximos passos sugeridos
- **Validação**: adicionar `KFold` (ou `RepeatedKFold`) e medir métricas por classe com intervalos.
- **Tuning**: Grid/Random Search para `C`, `penalty`, `solver` e tipo de `Scaler`.
- **Modelos**: testar `LinearSVC`, `RandomForest`, `XGBoost`/`LightGBM` (se disponíveis) via `OneVsRest`.
- **Thresholding**: calibrar limiar por classe (otimizar F1/Recall).
- **Balanceamento**: avaliar SMOTE/ADASYN quando disponível (cuidado com vazamento de dados).
- **Feature Engineering**: iterar nas features derivadas e checar importâncias/SHAP em modelos de árvore.
- **Export**: persistir `pipeline` com `joblib` para uso posterior (API de avaliação do bootcamp).


---

# Data Preparation — Manutenção Preditiva (CNC)

In [ ]:
import os, numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

DATA_RAW = "../data/raw"
DATA_INTERIM = "../data/interim"
os.makedirs(DATA_INTERIM, exist_ok=True)
df = pd.read_csv(os.path.join(DATA_RAW, "bootcamp_train.csv"))
df["torque_por_rpm"] = df["torque"] / df["velocidade_rotacional"].replace(0, np.nan)
target_cols = ["falha_maquina","FDF","FDC","FP","FTE","FA"]
X = df.drop(columns=target_cols)
Y = df[target_cols]
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()
num_pipe = Pipeline([("imp", SimpleImputer(strategy="median")), ("sc", StandardScaler())])
cat_pipe = Pipeline([("imp", SimpleImputer(strategy="most_frequent")), ("ohe", OneHotEncoder(handle_unknown="ignore"))])
preprocess = ColumnTransformer([("num", num_pipe, num_cols), ("cat", cat_pipe, cat_cols)])
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y["falha_maquina"])
X_train.to_csv(os.path.join(DATA_INTERIM,"X_train.csv"),index=False)
X_valid.to_csv(os.path.join(DATA_INTERIM,"X_valid.csv"),index=False)
y_train.to_csv(os.path.join(DATA_INTERIM,"y_train.csv"),index=False)
y_valid.to_csv(os.path.join(DATA_INTERIM,"y_valid.csv"),index=False)